

```
# Este es un proyecto de Modelo de IA de recetas de cocina
```



In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

recetas_ejemplo = [
    "Título: Pollo al Horno con Patatas. Ingredientes: 1 pollo entero, 500g patatas, sal, pimienta, aceite de oliva. Instrucciones: 1. Precalentar el horno a 200C. 2. Cortar las patatas. 3. Sazonar el pollo y las patatas. 4. Hornear durante 1 hora.",
    "Título: Ensalada César. Ingredientes: lechuga romana, croutones, queso parmesano, aderezo César. Instrucciones: 1. Lavar la lechuga. 2. Cortar la lechuga. 3. Mezclar todos los ingredientes con el aderezo.",
    "Título: Sopa de Lentejas. Ingredientes: 200g lentejas, 1 zanahoria, 1 cebolla, caldo de verduras, sal. Instrucciones: 1. Remojar las lentejas. 2. Picar las verduras. 3. Cocinar todo en el caldo hasta que las lentejas estén blandas.",
]

texto_completo = " ".join(recetas_ejemplo)

tokenizer = Tokenizer(num_words=None, oov_token="<unk>")
tokenizer.fit_on_texts([texto_completo])

word_index = tokenizer.word_index
print(f"Vocabulario tamaño: {len(word_index)}")

input_sequences = []
for receta in recetas_ejemplo:
    token_list = tokenizer.texts_to_sequences([receta])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])
padded_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

X = padded_sequences[:, :-1]
labels = padded_sequences[:, -1]

total_words = len(tokenizer.word_index) + 1
y = tf.keras.utils.to_categorical(labels, num_classes=total_words)

print(f"Número de secuencias de entrenamiento: {len(X)}")
print(f"Longitud máxima de secuencia: {max_sequence_len}")
print(f"Tamaño del vocabulario: {total_words}")

print("\nDefiniendo el modelo de generación de texto (LSTM)...")
model = Sequential([
    Embedding(total_words, 100, input_length=max_sequence_len-1),
    LSTM(150, return_sequences=True),
    Dropout(0.2),
    LSTM(100),
    Dropout(0.2),
    Dense(total_words, activation='softmax')
])

print("Compilando el modelo...")
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

print("\nEntrenando el modelo (esto tomará tiempo con datos reales)...")
try:
    history = model.fit(X, y, epochs=50, verbose=1)
except Exception as e:
    print(f"Error durante el entrenamiento: {e}")
    print("Asegúrate de que tus datos de entrada (X) y etiquetas (y) coinciden en dimensiones.")

def generate_recipe(seed_text, next_words, model, tokenizer, max_sequence_len):
    result = []
    current_text = seed_text
    result.append(seed_text)

    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([current_text])[0]
        padded_token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

        predicted_probabilities = model.predict(padded_token_list, verbose=0)[0]
        predicted_word_index = np.argmax(predicted_probabilities)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                output_word = word
                break
        current_text += " " + output_word
        result.append(output_word)

        if output_word in ['.', '!', '?']:
            break
    return " ".join(result)

print("\nGenerando una receta de ejemplo (¡la calidad depende del entrenamiento!):")
seed_text = "Título: Pastel de Chocolate. Ingredientes: chocolate, harina, huevos."
generated_recipe = generate_recipe(seed_text, 100, model, tokenizer, max_sequence_len)
print(generated_recipe)


Vocabulario tamaño: 60
Número de secuencias de entrenamiento: 101
Longitud máxima de secuencia: 40
Tamaño del vocabulario: 61

Definiendo el modelo de generación de texto (LSTM)...
Compilando el modelo...


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


Entrenando el modelo (esto tomará tiempo con datos reales)...
Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 98ms/step - accuracy: 0.0194 - loss: 4.1119 
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - accuracy: 0.0642 - loss: 4.0903 
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.0704 - loss: 4.0397 
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - accuracy: 0.0581 - loss: 3.9665 
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step - accuracy: 0.0704 - loss: 3.9325 
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - accuracy: 0.0838 - loss: 3.9045 
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - accuracy: 0.0611 - loss: 3.8588 
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step - accuracy: 0.0704 - loss: 3.8506
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 173ms/step - accuracy: 0.0867 - loss: 3.7801
Epoch 10/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - accuracy: 0.0888 - loss: 3.7755 
Epoch 11/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.1023 - loss: 3.6716 
Epoch 12/50